In [1]:
import numpy as np
import pandas as pd

In [3]:
v_cut = 4.8
n_v = 6
v_mesh = np.array([-v_cut + (2 * v_cut / n_v) * (i + 0.5) for i in range(n_v)])
borders = np.array([-v_cut + (2 * v_cut / n_v) * (i) for i in range(n_v + 1)])
print(v_mesh)
print(borders)

[-4.  -2.4 -0.8  0.8  2.4  4. ]
[-4.8 -3.2 -1.6  0.   1.6  3.2  4.8]


In [8]:
v_x = -2.4
(v_x + v_cut) / (2 * v_cut) * n_v + 0.5

2.0

In [21]:
v_x = -3.1
v_mesh[int(np.floor(n_v / 2 * (1 + v_x / v_cut)))]

-2.4

In [16]:
v_sq = v_mesh ** 2 * np.ones(n_v).reshape(-1, 1) + np.ones(n_v) * (v_mesh ** 2).reshape(-1, 1)
v_sq = v_sq * [v_sq <= 4.8**2]
v_sq


array([[[ 0.  , 21.76, 16.64, 16.64, 21.76,  0.  ],
        [21.76, 11.52,  6.4 ,  6.4 , 11.52, 21.76],
        [16.64,  6.4 ,  1.28,  1.28,  6.4 , 16.64],
        [16.64,  6.4 ,  1.28,  1.28,  6.4 , 16.64],
        [21.76, 11.52,  6.4 ,  6.4 , 11.52, 21.76],
        [ 0.  , 21.76, 16.64, 16.64, 21.76,  0.  ]]])

In [17]:
distr = np.exp(-0.5 * v_sq)
print(np.array_str(distr, precision=3))

[[[1.000e+00 1.883e-05 2.436e-04 2.436e-04 1.883e-05 1.000e+00]
  [1.883e-05 3.151e-03 4.076e-02 4.076e-02 3.151e-03 1.883e-05]
  [2.436e-04 4.076e-02 5.273e-01 5.273e-01 4.076e-02 2.436e-04]
  [2.436e-04 4.076e-02 5.273e-01 5.273e-01 4.076e-02 2.436e-04]
  [1.883e-05 3.151e-03 4.076e-02 4.076e-02 3.151e-03 1.883e-05]
  [1.000e+00 1.883e-05 2.436e-04 2.436e-04 1.883e-05 1.000e+00]]]


# collisions test

In [57]:
itop = pd.read_csv('col_test/index_to_p', sep='\t', header=None).to_numpy()[:, :-1].astype(int)

ptoi_x = np.zeros(itop.max() + 1)
ptoi_y = np.zeros(itop.max() + 1)
for i in range(itop.shape[0]):
    for j in range(itop.shape[1]):
        if itop[i, j] !=0:
            ptoi_x[itop[i, j]] = i + 1
            ptoi_y[itop[i, j]] = j + 1


print(itop.shape)

(20, 20)


In [15]:
cols = pd.read_csv('col_test/cols', sep='\t', index_col=None).drop(columns='trash')
interp = pd.read_csv('col_test/interp_active', sep='\t', index_col=None)
relv = pd.read_csv('col_test/relv_theta', sep='\t', index_col=None)

df_test = pd.concat([cols, relv, interp], axis=1)
del cols, interp, relv

print(df_test.shape)
df_test.head()

(50021, 17)


,v_a_x,v_a_y,v_b_x,v_b_y,b,eps,rel_v_x_new,rel_v_y_new,theta,is_active,p_alpha,p_beta,p_l,p_ls,p_m,p_ms,interp_r
0,-4.56,-2.64,-3.12,2.64,0.650407,-1,0.000000,0.00000,0.00000,0,0,0,0,0,0,0,0.000000e+00
1,-4.56,-0.24,-1.68,0.72,0.300814,-1,-0.266779,-3.02404,1.98054,1,3,91,42,58,36,21,1.927470e-15
2,-4.56,1.68,-0.24,-1.20,0.951220,1,0.000000,0.00000,0.00000,0,0,0,0,0,0,0,0.000000e+00
3,-4.56,4.08,1.20,-3.60,0.601627,1,0.000000,0.00000,0.00000,0,0,0,0,0,0,0,0.000000e+00
4,-4.56,-3.60,2.64,4.08,0.252034,-1,0.000000,0.00000,0.00000,0,0,0,0,0,0,0,0.000000e+00


In [67]:
row_num = 28
s = df_test.loc[row_num].astype(int)
print(ptoi_x[s.p_alpha], ptoi_y[s.p_alpha])
print(ptoi_x[s.p_beta], ptoi_y[s.p_beta])
print()
print(ptoi_x[s.p_l], ptoi_y[s.p_l])
print(ptoi_x[s.p_m], ptoi_y[s.p_m])
print()
print(ptoi_x[s.p_ls], ptoi_y[s.p_ls])
print(ptoi_x[s.p_ms], ptoi_y[s.p_ms])
print()

1.0 11.0
8.0 15.0

3.0 10.0
6.0 16.0

3.0 9.0
6.0 17.0



In [20]:
df_contr = df_test[['v_a_x', 'v_a_y', 'v_b_x', 'v_b_y', 'b', 'eps']]


In [ ]:
df_contr['is_active'] = ((df_contr.v_a_x ** 2 + df_contr.v_a_y ** 2 < 4.8 ** 2) & (df_contr.v_b_x ** 2 + df_contr.v_b_y ** 2 < 4.8 ** 2)).astype(int)

In [ ]:
df_contr['relv_x'] = df_contr.v_b_x - df_contr.v_a_x
df_contr['relv_y'] = df_contr.v_b_y - df_contr.v_a_y

df_contr['theta'] = 2 * np.arccos(np.sqrt(df_contr.b))

df_contr['rel_v_x_new'] = np.cos(df_contr.theta) * df_contr.relv_x - df_contr.eps * np.sin(df_contr.theta) * df_contr.relv_y
df_contr['rel_v_y_new'] = np.cos(df_contr.theta) * df_contr.relv_y + df_contr.eps * np.sin(df_contr.theta) * df_contr.relv_x

In [ ]:
df_contr['v_a_new_x'] = df_contr.v_b_x + df_contr.v_a_x - df_contr.rel_v_x_new / 2
df_contr['v_a_new_y'] = df_contr.v_b_y + df_contr.v_a_y - df_contr.rel_v_y_new / 2

In [38]:
df_contr = df_contr[df_contr.is_active == 1]
len(df_contr)

31182